In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [2]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")
train

,RowId,IntersectionId,Latitude,Longitude,EntryStreetName,ExitStreetName,EntryHeading,ExitHeading,Hour,Weekend,...,TimeFromFirstStop_p40,TimeFromFirstStop_p50,TimeFromFirstStop_p60,TimeFromFirstStop_p80,DistanceToFirstStop_p20,DistanceToFirstStop_p40,DistanceToFirstStop_p50,DistanceToFirstStop_p60,DistanceToFirstStop_p80,City
0,1921357,0,33.791659,-84.430032,Marietta Boulevard Northwest,Marietta Boulevard Northwest,NW,NW,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Atlanta
1,1921358,0,33.791659,-84.430032,Marietta Boulevard Northwest,Marietta Boulevard Northwest,SE,SE,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Atlanta
2,1921359,0,33.791659,-84.430032,Marietta Boulevard Northwest,Marietta Boulevard Northwest,NW,NW,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Atlanta
3,1921360,0,33.791659,-84.430032,Marietta Boulevard Northwest,Marietta Boulevard Northwest,SE,SE,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Atlanta
4,1921361,0,33.791659,-84.430032,Marietta Boulevard Northwest,Marietta Boulevard Northwest,NW,NW,2,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Atlanta
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856382,2777739,1990,39.938232,-75.149930,South 4th Street,Catharine Street,S,W,17,1,...,0.0,0.0,0.0,55.0,0.0,0.0,0.0,0.0,75.5,Philadelphia
856383,2777740,1990,39.938232,-75.149930,South 4th Street,South 4th Street,S,S,18,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Philadelphia
856384,2777741,1990,39.938232,-75.149930,South 4th Street,South 4th Street,S,S,19,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Philadelphia
856385,2777742,1990,39.938232,-75.149930,Catharine Street,Catharine Street,W,W,20,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Philadelphia


In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 856387 entries, 0 to 856386
Data columns (total 28 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   RowId                    856387 non-null  int64  
 1   IntersectionId           856387 non-null  int64  
 2   Latitude                 856387 non-null  float64
 3   Longitude                856387 non-null  float64
 4   EntryStreetName          848239 non-null  object 
 5   ExitStreetName           850100 non-null  object 
 6   EntryHeading             856387 non-null  object 
 7   ExitHeading              856387 non-null  object 
 8   Hour                     856387 non-null  int64  
 9   Weekend                  856387 non-null  int64  
 10  Month                    856387 non-null  int64  
 11  Path                     856387 non-null  object 
 12  TotalTimeStopped_p20     856387 non-null  float64
 13  TotalTimeStopped_p40     856387 non-null  float64
 14  Tota

## One-hot encoding

In [4]:
str_col = ['EntryStreetName', 'ExitStreetName', 'EntryHeading', 'ExitHeading', 'Path', 'City']
train_onehot = train.copy()
test_onehot = test.copy()
for col in str_col:
    head = train[col].value_counts().head(80).index.tolist()
    train_onehot[col] = train_onehot[col].apply(lambda x: 'pleaseRemove' if head.count(x) == 0 else x)
    test_onehot[col] = test_onehot[col].apply(lambda x: 'pleaseRemove' if head.count(x) == 0 else x)
    print(col)

# print(train['ExitStreetName'].value_counts())
# print(train['EntryHeading'].value_counts())
# print(train['ExitHeading'].value_counts())
# print(train['Path'].value_counts())
# print(train['City'].value_counts())

EntryStreetName
ExitStreetName
EntryHeading
ExitHeading
Path
City


In [5]:
X = pd.get_dummies(train_onehot, columns=['EntryStreetName', 'ExitStreetName', 'EntryHeading', 'ExitHeading', 'Path', 'City'])
Y = train[['TotalTimeStopped_p20', 'TotalTimeStopped_p50', 'TotalTimeStopped_p80', 'DistanceToFirstStop_p20', 'DistanceToFirstStop_p50', 'DistanceToFirstStop_p80']]

for col in X.columns.tolist():
    if col.find('pleaseRemove') != -1:
        X.drop(col, axis = 1, inplace = True)
    if col.find('Stop') != -1:
        X.drop(col, axis = 1, inplace = True)

KeyboardInterrupt: 

In [84]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 856387 entries, 0 to 856386
Columns: 342 entries, RowId to City_Philadelphia
dtypes: float64(17), int64(5), uint8(320)
memory usage: 405.1 MB


In [121]:
X.columns

Index(['RowId', 'IntersectionId', 'Latitude', 'Longitude', 'Hour', 'Weekend',
       'Month', 'EntryStreetName_10th Street Northwest',
       'EntryStreetName_17th Street Northwest',
       'EntryStreetName_Aramingo Avenue',
       ...
       'Path_West Lehigh Avenue_E_West Lehigh Avenue_E',
       'Path_West Lehigh Avenue_W_West Lehigh Avenue_W',
       'Path_West North Avenue_E_West North Avenue_E',
       'Path_West North Avenue_W_West North Avenue_W',
       'Path_West Roosevelt Road_E_West Roosevelt Road_E',
       'Path_West Roosevelt Road_W_West Roosevelt Road_W', 'City_Atlanta',
       'City_Boston', 'City_Chicago', 'City_Philadelphia'],
      dtype='object', length=267)

In [126]:
ml = RandomForestRegressor()
ml.fit(X, Y)

Intercept: 
 [  -458.30026973  -1435.10513984  -2611.01167796  -2465.5858137
  -9017.83200566 -25906.21103692]
Coefficients: 
 [[-3.32223662e-06  2.86787964e-04  5.67869243e+00 ...  9.41302429e+00
  -3.85153099e+01  1.18549590e+01]
 [-1.03522130e-05  7.70259451e-04  1.53383118e+01 ...  4.73927344e+01
  -1.26055266e+02  4.33562216e+01]
 [-1.90970973e-05  1.34381834e-03  3.02725961e+01 ...  7.11120636e+01
  -2.27633194e+02  7.11748381e+01]
 [-1.37122253e-05  9.69746236e-04  2.89988120e+01 ...  5.95464231e+01
  -2.08010684e+02  6.79639354e+01]
 [-4.06487058e-05  2.10898099e-03  1.08956609e+02 ...  1.93461823e+02
  -7.58972272e+02  2.39892987e+02]
 [-8.30875713e-05  4.84459533e-03  2.42772294e+02 ...  1.03870852e+03
  -2.28880353e+03  8.53383436e+02]]


In [122]:
predict_X = pd.get_dummies(test_onehot, columns=['EntryStreetName', 'ExitStreetName', 'EntryHeading', 'ExitHeading', 'Path', 'City'])

In [123]:
for col in predict_X.columns.tolist():
    if col.find('pleaseRemove') != -1:
        predict_X.drop(col, axis = 1, inplace = True)

In [136]:
predict_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1921357 entries, 0 to 1921356
Columns: 267 entries, RowId to City_Philadelphia
dtypes: float64(2), int64(5), uint8(260)
memory usage: 579.0 MB


In [130]:
predict_Y = ml.predict(predict_X)
predict_Y

array([[  8.08536983,  29.05174705,  64.90649015,  32.06752947,
        109.17722145, 272.97023099],
       [  8.83513208,  30.90587804,  67.84417561,  34.82890534,
        115.09218272, 288.14656464],
       [  8.1551219 ,  29.27766124,  65.99232919,  32.8318571 ,
        112.19282814, 278.21274667],
       ...,
       [  3.83238097,  13.20084416,  31.78148253,  17.24594878,
         63.40275366, 155.55650546],
       [  3.87179358,  13.7297705 ,  32.59336897,  16.95732811,
         61.83076012, 153.7441716 ],
       [  3.9015989 ,  13.83463767,  32.85267036,  17.0479291 ,
         62.15133291, 154.60190529]])

In [131]:
predict_Y.size

11528142

In [146]:
submission = pd.DataFrame(data = {'TargetId': [], 'Target': []})
for target_ID, v in enumerate(predict_Y):
    for index, j in enumerate(v):
        target_id = str(target_ID) + "_" + str(index)
        submission.append({'TargetId': target_id, 'Target': j}, ignore_index=True)

submission.to_csv("submission.csv", index=False)

TypeError: append() got an unexpected keyword argument 'ignore_idex'

In [3]:
train.columns

Index(['RowId', 'IntersectionId', 'Latitude', 'Longitude', 'EntryStreetName',
       'ExitStreetName', 'EntryHeading', 'ExitHeading', 'Hour', 'Weekend',
       'Month', 'Path', 'TotalTimeStopped_p20', 'TotalTimeStopped_p40',
       'TotalTimeStopped_p50', 'TotalTimeStopped_p60', 'TotalTimeStopped_p80',
       'TimeFromFirstStop_p20', 'TimeFromFirstStop_p40',
       'TimeFromFirstStop_p50', 'TimeFromFirstStop_p60',
       'TimeFromFirstStop_p80', 'DistanceToFirstStop_p20',
       'DistanceToFirstStop_p40', 'DistanceToFirstStop_p50',
       'DistanceToFirstStop_p60', 'DistanceToFirstStop_p80', 'City'],
      dtype='object')